# K‑Nearest Neighbors (KNN) Collaborative Filtering

- Load and validate the **subset_ratings.csv** file  
- Pivot into a **user × movie** ratings matrix (fill NA with 0)  
- Compute **item–item cosine similarity** and keep the top‑k neighbors  
- Predict unknown ratings by a **mean‑centered, similarity‑weighted average**  
- Split data per user (80 % train / 20 % test) and report **RMSE / MAE**  
- Output **Top‑N recommendations** for 1 000 sampled users to JSON  

In [2]:
import json, random, warnings
from pathlib import Path
from collections import defaultdict

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, lil_matrix
from sklearn.metrics import pairwise_distances, mean_squared_error, mean_absolute_error

warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
RATING_FILE = Path("subset_ratings.csv")
if not RATING_FILE.exists():
    raise FileNotFoundError(RATING_FILE)

ratings = pd.read_csv(RATING_FILE, usecols=["userId", "movieId", "rating"])
print("Loaded:", ratings.shape)

# map raw IDs → dense indices
uid2idx = {u: i for i, u in enumerate(ratings.userId.unique())}
mid2idx = {m: i for i, m in enumerate(ratings.movieId.unique())}
idx2mid = {i: m for m, i in mid2idx.items()}

ratings["u_idx"] = ratings.userId.map(uid2idx)
ratings["m_idx"] = ratings.movieId.map(mid2idx)

n_users, n_items = len(uid2idx), len(mid2idx)

Loaded: (2078625, 3)


In [4]:
def split_per_user(df, frac=0.2, seed=42):
    rng = np.random.default_rng(seed)
    train_mask = np.zeros(len(df), dtype=bool)
    for _, grp in df.groupby("userId"):
        idx = grp.index.to_numpy()
        rng.shuffle(idx)
        k = max(1, int(len(idx) * frac))
        train_mask[idx[k:]] = True
    return df[train_mask], df[~train_mask]

train_df, test_df = split_per_user(ratings, 0.20)
print(f"Train rows: {len(train_df)}   Test rows: {len(test_df)}")

Train rows: 1666865   Test rows: 411760


In [5]:
train_mat = csr_matrix(
    (train_df.rating.values,
     (train_df.m_idx, train_df.u_idx)),
    shape=(n_items, n_users)
)

item_sum    = np.asarray(train_mat.sum(axis=1)).ravel()
item_counts = np.diff(train_mat.indptr)                     # nnz per item
item_means  = np.divide(item_sum, item_counts,
                        out=np.zeros_like(item_sum, dtype=float),
                        where=item_counts != 0)

# subtract mean from each non-zero rating
train_mc = train_mat.copy().astype(np.float32)
for i in range(n_items):
    start, end = train_mc.indptr[i], train_mc.indptr[i + 1]
    train_mc.data[start:end] -= item_means[i]

In [6]:
item_sum    = np.asarray(train_mat.sum(axis=1)).ravel()
item_counts = np.diff(train_mat.indptr)           # nnz per row
item_means  = np.divide(item_sum, item_counts, out=np.zeros_like(item_sum), where=item_counts!=0)

# subtract mean from each non-zero element
train_mc = train_mat.copy().astype(np.float32)
for i in range(n_items):
    start, end = train_mc.indptr[i], train_mc.indptr[i+1]
    train_mc.data[start:end] -= item_means[i]

In [7]:
K = 40
print("Computing cosine similarity …")
cosine_dist = pairwise_distances(train_mc, metric="cosine", n_jobs=-1)
cosine_sim  = 1.0 - cosine_dist
np.fill_diagonal(cosine_sim, 0.0)

sim_mat = lil_matrix((n_items, n_items), dtype=np.float32)
for i in range(n_items):
    if K >= n_items:
        nbrs = np.argsort(cosine_sim[i])[::-1]
    else:
        nbrs = np.argpartition(cosine_sim[i], -K)[-K:]
        nbrs = nbrs[np.argsort(cosine_sim[i][nbrs])[::-1]]
    vals = cosine_sim[i, nbrs]
    mask = vals > 0
    if mask.any():
        sim_mat.rows[i]  = nbrs[mask].tolist()
        sim_mat.data[i]  = vals[mask].tolist()

sim_mat = sim_mat.tocsr()
sim_abs = sim_mat.copy();  sim_abs.data = np.abs(sim_abs.data)
del cosine_sim, cosine_dist

Computing cosine similarity …


In [8]:
user_rated = defaultdict(dict)
for r in train_df.itertuples():
    user_rated[r.userId][r.m_idx] = r.rating


In [9]:
user_rated = defaultdict(dict)
for r in train_df.itertuples():
    user_rated[r.userId][r.m_idx] = r.rating


In [10]:
def predict_single(uid, iid):
    """
    Mean-centred, similarity-weighted prediction for one (user, item).
    Uses O(K) operations.
    """
    numer = denom = 0.0
    rated = user_rated[uid]          # dict
    start, end = sim_mat.indptr[iid], sim_mat.indptr[iid + 1]
    for nbr_idx, sim in zip(sim_mat.indices[start:end], sim_mat.data[start:end]):
        if nbr_idx not in rated or sim <= 0:
            continue
        numer += sim * (rated[nbr_idx] - item_means[nbr_idx])
        denom += abs(sim)
    return item_means[iid] if denom == 0 else item_means[iid] + numer / denom


In [11]:
def top_n_for_user(uid, n=10):
    """
    Vectorised Top-N recommendation for a user using sparse mat-vec.
    """
    rated_dict = user_rated[uid]
    if not rated_dict:
        return []

    # sparse centred rating vector b (size n_items × 1)
    idxs = np.fromiter(rated_dict.keys(), dtype=int)
    vals = np.fromiter((r - item_means[i] for i, r in rated_dict.items()),
                       dtype=np.float32)
    b = csr_matrix((vals, (idxs, np.zeros_like(idxs))), shape=(n_items, 1))

    # similarity-weighted sums
    numer = (sim_mat @ b).toarray().ravel()
    denom = (sim_abs @ (b != 0)).toarray().ravel()

    scores = item_means + np.divide(numer, denom, out=np.zeros_like(numer),
                                    where=denom != 0)
    scores[idxs] = -np.inf                                # filter seen items
    top_idx = np.argpartition(scores, -n)[-n:][np.argsort(scores[np.argpartition(scores, -n)[-n:]])][::-1]
    return [idx2mid[int(i)] for i in top_idx]

In [12]:
y_true, y_pred = [], []
for row in test_df.itertuples():
    y_true.append(row.rating)
    y_pred.append(predict_single(row.userId, row.m_idx))

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae  = mean_absolute_error(y_true, y_pred)
print(f"RMSE = {rmse:.4f}   MAE = {mae:.4f}")

RMSE = 0.8401   MAE = 0.6312


In [13]:

N_USERS, TOP_N = 1_000, 10
sample_users = random.sample(
    train_df.userId.unique().tolist(),
    min(N_USERS, train_df.userId.nunique())
)

print("Generating recommendations …")
recs = {                                          # NumPy IDs may sneak in here
    int(u): top_n_for_user(u, TOP_N)              # keys cast to native int
    for u in sample_users
}

# Cast every movieId to a plain Python int so json can handle it
recs_py = {
    uid: [int(mid) for mid in mids]               # values cast to native int
    for uid, mids in recs.items()
}

PRED_DIR = Path("predictions")
PRED_DIR.mkdir(exist_ok=True)

OUTFILE = PRED_DIR / "knn_top10_subset.json"
with open(OUTFILE, "w") as f:
    json.dump(recs_py, f, indent=2)

print(f"✓ Saved {len(recs_py)} users × {TOP_N} recs → {OUTFILE}")

Generating recommendations …
✓ Saved 1000 users × 10 recs → predictions\knn_top10_subset.json


In [15]:
import json
from pathlib import Path
import random
from tqdm import tqdm

# assumes you already have:
#   – predict_single(uid, iid)  
#   – top_n_for_user(uid, n)  
#   – split_per_user(df, frac, seed)
#   – ratings, uid2idx, mid2idx, idx2mid

def generate_knn_preds(train_df, test_df, K=10):
    # build your train_mat, item_means, sim_mat, sim_abs, user_rated inside here,
    # exactly as in your existing code block above before “# RMSE/MAE”...
    #
    # then simply:
    recs = {}
    for u in tqdm(test_df.userId.unique(), desc="KNN preds"):
        recs[int(u)] = top_n_for_user(u, K)
    return recs

# make sure this directory exists
out_dir = Path("coldstart_pred")
out_dir.mkdir(exist_ok=True, parents=True)

for scenario in ("STANDARD", "USER", "ITEM"):
    print("→ KNN scenario:", scenario)
    # 1) load splits
    if scenario == "STANDARD":
        all_r = ratings  # your full subset_ratings.csv loaded earlier
        train_df, test_df = split_per_user(all_r, frac=0.2, seed=42)

    elif scenario == "USER":
        train_df = pd.read_csv("evaluation/user_cold_train.csv", usecols=["userId","movieId","rating"])
        test_df  = pd.read_csv("evaluation/user_cold_test.csv",  usecols=["userId","movieId","rating"])

    else:  # ITEM
        train_df = pd.read_csv("evaluation/item_cold_train.csv", usecols=["userId","movieId","rating"])
        test_df  = pd.read_csv("evaluation/item_cold_test.csv",  usecols=["userId","movieId","rating"])

    # 2) filter to known items (if you do the same in your other scripts)
    valid = set(mid2idx.keys())
    train_df = train_df[train_df.movieId.isin(valid)].reset_index(drop=True)
    test_df  = test_df [test_df .movieId.isin(valid)].reset_index(drop=True)

    # 3) rebuild all of KNN’s internal data: train_mat, item_means, sim_mat, sim_abs, user_rated
    #    (just copy‐paste the corresponding blocks from above)
    #
    #    e.g.:
    #    train_mat = csr_matrix(...)
    #    compute item_means, train_mc, cosine_sim, sim_mat, sim_abs, user_rated

    # 4) generate & dump
    preds = generate_knn_preds(train_df, test_df, K=10)
    # convert both keys and all movie‐ids to native Python ints
    clean_preds = {
        int(u): [int(m) for m in mids]
        for u, mids in preds.items()
    }

    outfn = out_dir / f"knn_{scenario.lower()}_top10.json"
    with open(outfn, "w") as fp:
        json.dump(clean_preds, fp, indent=2)

    print(f"  → saved {len(preds)} users → {outfn}\n")


→ KNN scenario: STANDARD


KNN preds: 100%|██████████| 10000/10000 [00:15<00:00, 652.51it/s]


  → saved 10000 users → coldstart_pred\knn_standard_top10.json

→ KNN scenario: USER


KNN preds: 100%|██████████| 500/500 [00:00<00:00, 660.52it/s]


  → saved 500 users → coldstart_pred\knn_user_top10.json

→ KNN scenario: ITEM


KNN preds: 100%|██████████| 7982/7982 [00:12<00:00, 649.10it/s]

  → saved 7982 users → coldstart_pred\knn_item_top10.json

